In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

from pt_runner.mlp import CheckpointHandler, DataHandlerPT, EarlyStopper


In [ ]:
# Listing saved files (if needed)
CheckpointHandler.list_saved_files()

In [ ]:
# New run
NEW_RUN = True
DT_REF = None

# Resuming
# NEW_RUN = False
# DT_REF = "2025-05-30_08-36"

In [ ]:
RANDOM_STATE = 0

In [ ]:
df = pd.read_excel("data.xlsx", index_col="exp")
_X = df.iloc[:, :-3].values
_Y = df.iloc[:, -3:].values
print(_X.shape)
print(_Y.shape)
data_handler = DataHandlerPT(
    _X=_X, _Y=_Y, scalerX=StandardScaler(), scalerY=StandardScaler()
)

In [ ]:
# Define the model
num_features = data_handler._X.shape[1]
num_outputs = data_handler._Y.shape[1]


class MyModel(nn.Module):
    def __init__(self, num_features, num_outputs):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 24)
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, 6)
        self.fc4 = nn.Linear(6, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = MyModel(num_features, num_outputs)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=5)
loss_fn = nn.MSELoss()  # mean square error

In [ ]:
from torchinfo import summary

input_size = (num_features,)
summary(model, input_size=input_size)

- `torch.optim.lr_scheduler.ReduceLROnPlateau`
  - This is a special scheduler that monitors a validation metric (often the validation loss).
  - If that metric hasn't improved for a certain number of "patience" epochs, it automatically reduces (lowers) the learning rate.
- `tqdm(...)`
  - `tqdm` is a library that wraps over an iterator (like range) to provide a progress bar in your console. This way, you can see how far you are in your training!
- `torch.nn.utils.clip_grad_norm_(...)`
  - During training, especially with deep networks, sometimes gradients can become very large (this is called the exploding gradients problem).
  - This can destabilize or even ruin training.
  - By "clipping"—or limiting—the gradients so the overall "norm" is at most 1.0, you ensure that updates to your model aren’t too large and that training stays stable.
- `loss.item() * X_batch.size(0)`
  - Multiplies the average loss per sample by the number of samples in the batch to get the total loss for this batch.
- Tensorboard
  - `tensorboard --logdir=src/T02_mlp/runs`
  - Visit http://localhost:6006
  - `remove-item ./src/T02_mlp/runs -Force`


In [ ]:
n_epochs = 1000  # number of epochs to run
batch_size = 10  # size of each batch
validation_interval = 10  # Evaluate every 10 epochs
log_name = "M1"

# Save/load
cph = CheckpointHandler()
cph.make_dir("./checkpoints")
if NEW_RUN:
    dt = cph.get_dt()
    log_dir = f"runs/{dt}"
    save_path = f"./checkpoints/{dt}.pth"
    epoch_start = 0
else:
    log_dir = f"runs/{DT_REF}"
    load_path = f"./checkpoints/{DT_REF}.pth"
    save_path = load_path
    model, optimizer, epoch, val_loss = cph.load(
        load_path=load_path, model=model, optimizer=optimizer
    )
    epoch_start = epoch
    print(f"Resuming from epoch: {epoch_start}")

epoch_end = epoch_start + n_epochs

# Initialize Components
early_stopper = EarlyStopper(patience=10)
writer = SummaryWriter(log_dir=log_dir)

# Data
data_handler.split_and_scale(test_size=0.2, val_size=0.1, random_state=RANDOM_STATE)
ds_train = data_handler.get_train()
ds_val = data_handler.get_val()
loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False)

# Main loop
for epoch in tqdm(
    range(epoch_start, epoch_end), initial=epoch_start, desc="Epoch", total=n_epochs
):
    # Training Phase
    model.train()
    epoch_train_loss = 0.0

    for X_batch, Y_batch in loader_train:
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch)
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        # Update weights
        optimizer.step()
        # Multiplies the average loss per sample by the number of
        # samples in the batch to get the total loss for this batch.
        epoch_train_loss += loss.item() * X_batch.size(0)

    avg_train_loss = epoch_train_loss / len(loader_train.dataset)

    # Validation Phase
    if epoch % validation_interval == 0 or epoch == epoch_start:
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, Y_val in loader_val:
                Y_pred = model(X_val)
                val_loss += loss_fn(Y_pred, Y_val).item() * X_val.size(0)

        avg_val_loss = val_loss / len(loader_val.dataset)
        scheduler.step(avg_val_loss)

        # Early Stopping and Checkpoint
        es = early_stopper(avg_val_loss)
        if es["best_loss"]:
            cph.save(
                save_path=save_path,
                model=model,
                optimizer=optimizer,
                val_loss=avg_val_loss,
                epoch=epoch,
            )
            print("Save model @ epoch:", epoch)
        if es["early_stop"]:
            print("Stopped at epoch:", epoch)
            break

    writer.add_scalars(
        log_name, {"train_loss": avg_train_loss, "val_loss": avg_val_loss}, epoch
    )
